In [2]:
# Run this to ensure TensorFlow 2.x is used
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

In [3]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/john/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

In [6]:
vocab_size = 5000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8

In [13]:
articles = []
labelsList = []

#raw_data = pd.read_csv('/content/drive/My Drive/new_transaction.csv')

with open("isear.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        labelsList.append(row[0])
        article = row[0]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)
print(len(labelsList))
print(len(articles))

7666
7666


In [42]:
import pandas as pd

data = pd.read_csv("isear.csv", delimiter="|" , error_bad_lines=False)

b'Skipping line 199: expected 43 fields, saw 44\nSkipping line 222: expected 43 fields, saw 44\nSkipping line 248: expected 43 fields, saw 44\nSkipping line 350: expected 43 fields, saw 44\nSkipping line 395: expected 43 fields, saw 44\nSkipping line 423: expected 43 fields, saw 44\nSkipping line 480: expected 43 fields, saw 44\nSkipping line 552: expected 43 fields, saw 44\nSkipping line 601: expected 43 fields, saw 45\nSkipping line 604: expected 43 fields, saw 45\nSkipping line 612: expected 43 fields, saw 45\nSkipping line 722: expected 43 fields, saw 44\nSkipping line 765: expected 43 fields, saw 44\nSkipping line 800: expected 43 fields, saw 46\nSkipping line 818: expected 43 fields, saw 45\nSkipping line 891: expected 43 fields, saw 44\nSkipping line 928: expected 43 fields, saw 44\nSkipping line 963: expected 43 fields, saw 44\nSkipping line 965: expected 43 fields, saw 45\nSkipping line 974: expected 43 fields, saw 44\nSkipping line 985: expected 43 fields, saw 44\nSkipping li

In [33]:
data.head()

,ID,CITY,COUN,SUBJ,SEX,AGE,RELI,PRAC,FOCC,MOCC,...,RELA,VERBAL,NEUTRO,Field1,Field3,Field2,MYKEY,SIT,STATE,Unnamed: 42
0,11001,1,1,1,1,33,1,2,6,1,...,3,2,0,joy,4,3,110011,"During the period of falling in love, each tim...",1,NaN
1,11001,1,1,1,1,33,1,2,6,1,...,2,0,0,fear,3,2,110012,When I was involved in a traffic accident.,1,NaN
2,11001,1,1,1,1,33,1,2,6,1,...,1,0,0,anger,1,3,110013,When I was driving home after several days of...,1,NaN
3,11001,1,1,1,1,33,1,2,6,1,...,1,0,2,sadness,4,4,110014,When I lost the person who meant the most to me.,1,NaN
4,11001,1,1,1,1,33,1,2,6,1,...,2,0,0,disgust,4,4,110015,The time I knocked a deer down - the sight of ...,1,NaN


In [41]:
#data.drop(['ID', 'CITY', 'COUN', 'SUBJ','SEX', 'PRAC', 'MOCC','AGE','RELI','FOCC','FIEL'], axis = 1) 

# Remove all columns between column index 1 to 19
data.drop(data.iloc[:, 1:19], inplace = True, axis = 1)
data.head()

,ID
0,11001
1,11001
2,11001
3,11001
4,11001


In [115]:
df2 = data.loc[:,'Field1':'SIT']
df2 = df2.drop(['Field3', 'Field2', 'MYKEY'], axis = 1)
#df2.head()

emotion_list = df2['Field1'].tolist()
sentence_list = df2['SIT'].tolist()
#Field1 to emotionlist, SIT to sentenceList, 

print(len(sentence_list))
#articless = []

# for each sentence in sentence list, I wnat to check if it contains a stopword, if yes replace the word with ' '
#for sentence in sentence_list:
 #   for word in STOPWORDS:
  #      token = ' ' + word + ' '
   #     sentence = sentence.replace(token, ' ')
    #    sentence = sentence.replace(' ', ' ')
    #sentence_list.append(sentence) 

#print(len(sentence_list))

print(sentence_list[0:3])

7503
['During the period of falling in love, each time that we met and á especially when we had not met for a long time.', 'When I was involved in a traffic accident.', 'When I was driving home after  several days of hard work, there á was a motorist ahead of me who was driving at 50 km/hour and á refused, despite his low speeed to let me overtake.']


In [116]:
for word in STOPWORDS:
    token = ' ' + word + ' '
    res = list(map(lambda token: str.replace(token, ' ', ' '), sentence_list))
print(res[0:3])

['During the period of falling in love, each time that we met and á especially when we had not met for a long time.', 'When I was involved in a traffic accident.', 'When I was driving home after  several days of hard work, there á was a motorist ahead of me who was driving at 50 km/hour and á refused, despite his low speeed to let me overtake.']


In [119]:
sentenceFiltered = []
for w in sentence_list:
    if w not in STOPWORDS:
        sentenceFiltered.append(w)
len(sentenceFiltered)        

7503

In [108]:
ress = [map(lambda word: str.replace(word, ' ', ' '), sentence_list) for word in STOPWORDS]
ad = [list(map(lambda word: str.replace(word, ' ', ' '), sentence_list)) for word in STOPWORDS]
len(ad)

179

you'll
yourself
here
who
aren't
them
couldn't
being
until
those
isn
do
needn
than
to
out
herself
a
shan't
be
any
all
you're
how
s
was
yours
this
haven't
most
these
myself
hadn't
off
weren
have
can
wouldn
other
ours
she's
up
our
been
hasn't
mightn't
isn't
does
y
shouldn
between
d
as
or
about
now
doesn't
into
nor
there
m
aren
he
further
didn't
am
are
won't
weren't
didn
won
haven
very
if
having
me
more
ll
during
after
from
too
because
wasn't
you'd
whom
down
hadn
over
its
o
should've
shan
then
where
by
ma
why
same
that
couldn
the
shouldn't
on
doesn
is
your
under
you've
few
of
above
below
will
were
hasn
she
itself
so
re
such
both
wasn
themselves
it's
once
has
when
before
his
which
only
an
with
mustn
while
yourselves
what
himself
needn't
i
it
doing
mightn
some
and
ain
ve
ourselves
not
for
t
just
did
we
no
don
theirs
own
they
at
you
him
their
against
in
mustn't
my
should
that'll
through
wouldn't
hers
but
again
had
each
don't
her
